1. Muhammad Luthfi Shahab
2. luthfi.shahab@gmail.com
3. 100060503@ku.ac.ae

Open 'blend_schedule_result.xlsx' to see the result.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters

In [2]:
blend_quantity = 1000
quantity_threshold = 1000
transfer_time_from_blending = 16 + 27 # from warehouse to blending + from blending to silo

In [3]:
wastage = 0

In [4]:
# For storing blending schedule/table
data = {
    "blend_time": [],
    "blend_code": [],
    "blend_desc": [], # description
    "sequence": [],
    "quantity": [],
    "silo": [],
    "line": [],
    "sku_code": [],
    "quantity_line": [],
}

blend_schedule = pd.DataFrame(data)

# Load Data

In [5]:
blend_stock1 = pd.read_excel('blend_stock1.xlsx')
blend_stock1 = blend_stock1.set_index('location')
blend_stock1

,blend_code,blend_description,quantity
location,,,
siloA,B0004,Loose Tea 1,5000
siloB,B0004,Loose Tea 1,4000
siloC,B0001,Black Tea 1,6000
siloD,B0001,Black Tea 1,3000
siloE,B0002,Black Tea 2,3000
siloF,B0003,Black Tea 3,4500


In [6]:
consumption_rate = pd.read_excel('consumption_rate.xlsx')
consumption_rate = consumption_rate.set_index('line')
consumption_rate

,icr,efficiency
line,,
line1,541,0.80
line2,168,0.71
line3,168,0.71
line4,1800,0.83
line5,1536,0.85
line6,640,0.83
line7,246,0.80
line8,640,0.80
line9,162,0.80


In [7]:
production_schedule = pd.read_excel('production_schedule.xlsx')
production_schedule = production_schedule.set_index('sku_code')
production_schedule

,line,description,blend_code,blend_description,quantity,sequence
sku_code,,,,,,
F0001,line1,LYL 1,B0001,Black Tea 1,30780,1
F0027,line2,ENV 3,B0012,Black Tea 7,1302,1
F0023,line3,ENV 1,B0009,Black Tea 4,233,1
F0024,line3,ENV 2,B0010,Black Tea 5,3726,2
F0014,line4,LYL Loose 3,B0004,Loose Tea 1,11328,1
F0015,line4,LYL Loose 4,B0004,Loose Tea 1,13680,2
F0016,line4,BB Loose 6,B0005,Loose Tea 2,28800,3
F0017,line4,BB Loose 7,B0006,Loose Tea 3,17820,4
F0018,line4,BB Loose 8,B0006,Loose Tea 3,3240,5


In [8]:
time_expected = np.zeros(production_schedule.shape[0], dtype=int)
time_last = np.zeros(production_schedule.shape[0], dtype=int)
for i in range(production_schedule.shape[0]):
    line = production_schedule.iloc[i]['line']
    icr = consumption_rate.loc[line]['icr']
    efficiency = consumption_rate.loc[line]['efficiency']
    time_expected[i] = (int)(60*production_schedule.iloc[i]['quantity']/(icr*efficiency))
    
production_schedule['time_expected'] = time_expected
production_schedule['time_last'] = time_last
production_schedule

,line,description,blend_code,blend_description,quantity,sequence,time_expected,time_last
sku_code,,,,,,,,
F0001,line1,LYL 1,B0001,Black Tea 1,30780,1,4267,0
F0027,line2,ENV 3,B0012,Black Tea 7,1302,1,654,0
F0023,line3,ENV 1,B0009,Black Tea 4,233,1,117,0
F0024,line3,ENV 2,B0010,Black Tea 5,3726,2,1874,0
F0014,line4,LYL Loose 3,B0004,Loose Tea 1,11328,1,454,0
F0015,line4,LYL Loose 4,B0004,Loose Tea 1,13680,2,549,0
F0016,line4,BB Loose 6,B0005,Loose Tea 2,28800,3,1156,0
F0017,line4,BB Loose 7,B0006,Loose Tea 3,17820,4,715,0
F0018,line4,BB Loose 8,B0006,Loose Tea 3,3240,5,130,0


In [9]:
rm_stock = pd.read_excel('rm_stock.xlsx')
rm_stock = rm_stock.set_index('rm_code')
rm_stock

,blend_code,blend_description,rm_description,quantity,location
rm_code,,,,,
R0001,B0001,Black Tea 1,Raw: Black Tea 1,60000,Warehouse
R0002,B0002,Black Tea 2,Raw: Black Tea 2,20000,Warehouse
R0003,B0003,Black Tea 3,Raw: Black Tea 3,50000,Warehouse
R0004,B0004,Loose Tea 1,Raw: Loose Tea 1,50000,Warehouse
R0005,B0005,Loose Tea 2,Raw: Loose Tea 2,30000,Warehouse
R0006,B0006,Loose Tea 3,Raw: Loose Tea 3,45000,Warehouse
R0007,B0007,Loose Tea 4,Raw: Loose Tea 4,40000,Warehouse
R0008,B0008,Loose Tea 5,Raw: Loose Tea 5,15000,Warehouse
R0009,B0009,Black Tea 4,Raw: Black Tea 4,0,Warehouse


In [10]:
blend = pd.read_excel('blend.xlsx')
blend = blend.set_index('blend_code')
blend

,blend_description,rm_code
blend_code,,
B0001,Black Tea 1,R0001
B0002,Black Tea 2,R0002
B0003,Black Tea 3,R0003
B0004,Loose Tea 1,R0004
B0005,Loose Tea 2,R0005
B0006,Loose Tea 3,R0006
B0007,Loose Tea 4,R0007
B0008,Loose Tea 5,R0008
B0009,Black Tea 4,R0009


In [11]:
next_sku_code = pd.read_excel('next_sku_code.xlsx')
next_sku_code = next_sku_code.set_index('sku_code')
next_sku_code

,sku_code_next,blend,blend_next
sku_code,,,
F0014,F0015,B0004,B0004
F0015,F0016,B0004,B0005
F0016,F0017,B0005,B0006
F0017,F0018,B0006,B0006
F0018,F0019,B0006,B0006
F0019,F0020,B0006,B0008
F0020,F0021,B0008,B0008
F0021,F0022,B0008,B0007
F0007,F0008,B0004,B0004


In [12]:
transfer_time1 = pd.read_excel('transfer_time3.xlsx')
transfer_time1 = transfer_time1.set_index('from')
transfer_time1

,line1,line2,line3,line4,line5,line6,line7,line8,line9,line10,line11
from,,,,,,,,,,,
siloA,NaN,NaN,NaN,NaN,275.0,NaN,NaN,NaN,NaN,NaN,NaN
siloB,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
siloC,831.0,3018.0,3018.0,NaN,NaN,677.0,1829.0,704.0,2778.0,1389.0,831.0
siloD,831.0,3018.0,3018.0,NaN,NaN,677.0,1829.0,704.0,2778.0,1389.0,831.0
siloE,831.0,3018.0,3018.0,NaN,NaN,677.0,1829.0,704.0,2778.0,1389.0,831.0
siloF,831.0,3018.0,3018.0,NaN,NaN,677.0,1829.0,704.0,2778.0,1389.0,831.0


# Functions

In [13]:
def solve_connection_matrix(connection_matrix):
    sku_codes = connection_matrix.index
    silos = connection_matrix.columns

    Iter = 0
    while connection_matrix.sum().sum() > 0:
        Iter = Iter + 1
        print('Iter = ' + str(Iter))
        print()

        # For storing times to empty the silos
        times = np.zeros(connection_matrix.shape[0])

        # Checking all elements in connection matrix
        i = 0
        for sku_code in sku_codes:
            for silo in silos:
                # If it is equal to 1, it means a silo is connected to sku_code/job
                if connection_matrix.loc[sku_code][silo] == 1:
                    if production_schedule.loc[sku_code]['quantity'] > 0:
                        # Calculate the time to empty this silo
                        line = production_schedule.loc[sku_code]['line']
                        times[i] = times[i] + 60*blend_stock1.loc[silo]['quantity']/(consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency'])

                        # Updating the silos / Substracting the stock with the amount sended to a line
                        qty1 = math.floor(transfer_time_from_blending*consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency']/60)
                        qty2 = blend_stock1.loc[silo]['quantity']
                        qty3 = production_schedule.loc[sku_code]['quantity']
                        qty_min = min(qty1, qty2, qty3)
                        blend_stock1.at[silo, 'quantity'] = blend_stock1.loc[silo]['quantity'] - qty_min

                        # Updating the overall needed quantity
                        production_schedule.at[sku_code, 'quantity'] = production_schedule.loc[sku_code]['quantity'] - qty_min
                        production_schedule.at[sku_code, 'time_last'] = Iter*transfer_time_from_blending
                        print('Stock needed for ' + sku_code + ' = ' + str(production_schedule.loc[sku_code]['quantity']))
                        print()
                    else:
                        connection_matrix.at[sku_code, silo] = 0
                        print('Connection Matrix Changed')
                        print()
                        print(connection_matrix)
            i = i + 1

        # Break if connection_matrix is empty
        if connection_matrix.sum().sum() == 0:
            break

        for i in range(connection_matrix.shape[0]):
            sku_code = sku_codes[i]
            if sum(connection_matrix.loc[sku_code]) == 0:
                times[i] = 1000000

        # Finding the fastest silo to empty
        print(times.astype(int))
        print()
        index = np.argmin(times)
        sku_code = sku_codes[index]

        blend_code = production_schedule.loc[sku_code]['blend_code']

        destination1 = '';
        for i in range(connection_matrix.shape[1]):
            if connection_matrix.loc[sku_code].iloc[i] == 1:
                destination1 = connection_matrix.columns[i]

        line = production_schedule.loc[sku_code]['line']

        if blend_stock1.loc[destination1]['quantity'] < quantity_threshold:
            # Add a new row into the blend schedule/table
            blend_schedule.loc[len(blend_schedule.index)] = [Iter, blend_code, blend.loc[blend_code]['blend_description'], production_schedule.loc[sku_code]['sequence'], blend_quantity, destination1, line, sku_code, production_schedule.loc[sku_code]['quantity']]
            print(blend_schedule.tail(5))
            print()

            # Adding a new stock to the silo with fastest time to empty
            blend_stock1.at[destination1, 'quantity'] = blend_stock1.at[destination1, 'quantity'] + blend_quantity
            print(blend_stock1)

            # Updating raw material
            rm_code = blend.loc[blend_code]['rm_code']
            rm_stock.at[rm_code, 'quantity'] = rm_stock.loc[rm_code]['quantity'] - blend_quantity

        print()
        print('========================================================================================================================')
        print()

In [14]:
def solve_connection_matrix1(connection_matrix, Iter0):
    sku_codes = connection_matrix.index
    silos = connection_matrix.columns

    Iter = Iter0
    while connection_matrix.sum().sum() > 0:
        Iter = Iter + 1
        print('Iter = ' + str(Iter))
        print()
        
        # qty_sended_to_lines
        qty_sended_to_lines = 0;

        # For storing times to empty the silos
        times = np.zeros(connection_matrix.shape[0])

        # Checking all elements in connection matrix
        i = 0
        for sku_code in sku_codes:
            for silo in silos:
                # If it is equal to 1, it means a silo is connected to sku_code/job
                if connection_matrix.loc[sku_code][silo] == 1:
                    if production_schedule.loc[sku_code]['quantity'] > 0:
                        # Calculate the time to empty this silo
                        line = production_schedule.loc[sku_code]['line']
                        times[i] = times[i] + 60*blend_stock1.loc[silo]['quantity']/(consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency'])

                        # Updating the stock / Substracting the stock with the amount sended to a line
                        qty1 = math.floor(transfer_time_from_blending*consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency']/60)
                        qty2 = blend_stock1.loc[silo]['quantity']
                        qty3 = production_schedule.loc[sku_code]['quantity']
                        qty_min = min(qty1, qty2, qty3)
                        blend_stock1.at[silo, 'quantity'] = blend_stock1.loc[silo]['quantity'] - qty_min
                        qty_sended_to_lines = qty_sended_to_lines + qty_min

                        # Updating the overall needed quantity
                        production_schedule.at[sku_code, 'quantity'] = production_schedule.loc[sku_code]['quantity'] - qty_min
                        print('Stock needed for ' + sku_code + ' = ' + str(production_schedule.loc[sku_code]['quantity']))
                        print()
                    else:
                        connection_matrix.at[sku_code, silo] = 0
                        print('Connection Matrix is Changed')
                        print()
                        print(connection_matrix)
            i = i + 1

        # Break if stock in silos are empty
        if qty_sended_to_lines == 0:
            break
            
        # Break if connection_matrix is empty
        if connection_matrix.sum().sum() == 0:
            break

        for i in range(connection_matrix.shape[0]):
            sku_code = sku_codes[i]
            if sum(connection_matrix.loc[sku_code]) == 0:
                times[i] = 1000000

        # Finding the fastest time to empty
        print(times.astype(int))
        print()
        index = np.argmin(times)
        sku_code = sku_codes[index]

        blend_code = production_schedule.loc[sku_code]['blend_code']

        destination1 = '';
        for i in range(connection_matrix.shape[1]):
            if connection_matrix.loc[sku_code].iloc[i] == 1:
                destination1 = connection_matrix.columns[i]

        line = production_schedule.loc[sku_code]['line']

        if blend_stock1.loc[destination1]['quantity'] < quantity_threshold:
            # Checking raw material
            blend_code = production_schedule.loc[sku_code]['blend_code']
            rm_code = blend.loc[blend_code]['rm_code']
            if rm_stock.loc[rm_code]['quantity'] > blend_quantity:
                # Add a new row into the blend schedule/table
                blend_schedule.loc[len(blend_schedule.index)] = [Iter, blend_code, blend.loc[blend_code]['blend_description'], production_schedule.loc[sku_code]['sequence'], blend_quantity, destination1, line, sku_code, production_schedule.loc[sku_code]['quantity']]
                print(blend_schedule.tail(5))
                print()

                # Adding a new stock to the silo with fastest time to empty
                blend_stock1.at[destination1, 'quantity'] = blend_stock1.at[destination1, 'quantity'] + blend_quantity
                print(blend_stock1)

                # Updating raw material
                rm_code = blend.loc[blend_code]['rm_code']
                rm_stock.at[rm_code, 'quantity'] = rm_stock.loc[rm_code]['quantity'] - blend_quantity
            else:
                print()

        print()
        print('========================================================================================================================')
        print()
        
    return (Iter-1)

In [15]:
def solve_connection_matrix2(connection_matrix, Iter0):
    sku_codes = connection_matrix.index
    silos = connection_matrix.columns
    
    # Stop if connection matrix is changed
    is_connection_matrix_changed = 0

    Iter = Iter0
    while connection_matrix.sum().sum() > 0:
        Iter = Iter + 1
        print('iter = ' + str(Iter))
        print()

        # For storing times to empty the silos
        times = np.zeros(connection_matrix.shape[0])

        # Checking all elements in connection matrix
        i = 0
        for sku_code in sku_codes:
            for silo in silos:
                # If it is equal to 1, it means a silo is connected to sku_code/job
                if connection_matrix.loc[sku_code][silo] == 1:
                    if production_schedule.loc[sku_code]['quantity'] > 0:
                        # Calculate the time to empty this silo
                        line = production_schedule.loc[sku_code]['line']
                        times[i] = times[i] + 60*blend_stock1.loc[silo]['quantity']/(consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency'])

                        # Updating the stock / Substracting the stock with the amount sended to a line
                        qty1 = math.floor(transfer_time_from_blending*consumption_rate.loc[line]['icr']*consumption_rate.loc[line]['efficiency']/60)
                        qty2 = blend_stock1.loc[silo]['quantity']
                        qty3 = production_schedule.loc[sku_code]['quantity']
                        qty_min = min(qty1, qty2, qty3)
                        blend_stock1.at[silo, 'quantity'] = blend_stock1.loc[silo]['quantity'] - qty_min

                        # Updating the overall needed quantity
                        production_schedule.at[sku_code, 'quantity'] = production_schedule.loc[sku_code]['quantity'] - qty_min
                        print('Stock needed for ' + sku_code + ' = ' + str(production_schedule.loc[sku_code]['quantity']))
                        print()
                    else:
                        connection_matrix.at[sku_code, silo] = 0
                        is_connection_matrix_changed = 1
                        print('Connection Matrix is Changed')
                        print()
                        print(connection_matrix)
            i = i + 1
        
        # Stop if connection matrix is changed
        if is_connection_matrix_changed == 1:
            break

        # Break if connection_matrix is empty
        if connection_matrix.sum().sum() == 0:
            break

        for i in range(connection_matrix.shape[0]):
            sku_code = sku_codes[i]
            if sum(connection_matrix.loc[sku_code]) == 0:
                times[i] = 1000000

        # Finding the fastest time to empty
        print(times.astype(int))
        print()
        index = np.argmin(times)
        sku_code = sku_codes[index]

        blend_code = production_schedule.loc[sku_code]['blend_code']

        destination1 = '';
        for i in range(connection_matrix.shape[1]):
            if connection_matrix.loc[sku_code].iloc[i] == 1:
                destination1 = connection_matrix.columns[i]

        line = production_schedule.loc[sku_code]['line']

        if blend_stock1.loc[destination1]['quantity'] < quantity_threshold:
            # Add a new row into the blend schedule/table
            blend_schedule.loc[len(blend_schedule.index)] = [Iter, blend_code, blend.loc[blend_code]['blend_description'], production_schedule.loc[sku_code]['sequence'], blend_quantity, destination1, line, sku_code, production_schedule.loc[sku_code]['quantity']]
            print(blend_schedule.tail(5))
            print()

            # Adding a new stock to the silo with fastest time to empty
            blend_stock1.at[destination1, 'quantity'] = blend_stock1.at[destination1, 'quantity'] + blend_quantity
            print(blend_stock1)

            # Updating raw material
            rm_code = blend.loc[blend_code]['rm_code']
            rm_stock.at[rm_code, 'quantity'] = rm_stock.loc[rm_code]['quantity'] - blend_quantity
            
            if rm_stock.loc[rm_code]['quantity'] == 0:
                print()
                print('EMPTY STOCK IN THE WAREHOUSE.')
                print('EMPTY STOCK IN THE WAREHOUSE.')
                print('EMPTY STOCK IN THE WAREHOUSE.')
                print('EMPTY STOCK IN THE WAREHOUSE.')
                print('EMPTY STOCK IN THE WAREHOUSE.')
                print()
                return (Iter)

        print()
        print('========================================================================================================================')
        print()
        
    return (Iter-1)

# Iteration Process for Silo C, D, E, F

In [16]:
connection_matrix = pd.read_excel('connection_matrix_CDEF1.xlsx')
connection_matrix = connection_matrix.set_index('sku_code')
connection_matrix

,siloC,siloD,siloE,siloF
sku_code,,,,
F0001,1,0,0,0
F0004,0,0,1,0
F0002,0,0,1,0
F0006,0,1,0,0
F0028,0,0,0,1


In [17]:
solve_connection_matrix(connection_matrix)

Iter = 1

Stock needed for F0001 = 30470

Stock needed for F0004 = 1081

Stock needed for F0002 = 19060

Stock needed for F0006 = 13899

Stock needed for F0028 = 16835

[831 351 297 914 623]



Iter = 2

Stock needed for F0001 = 30160

Stock needed for F0004 = 715

Stock needed for F0002 = 18680

Stock needed for F0006 = 13758

Stock needed for F0028 = 16525

[788 264 213 871 580]



Iter = 3

Stock needed for F0001 = 29850

Stock needed for F0004 = 349

Stock needed for F0002 = 18300

Stock needed for F0006 = 13617

Stock needed for F0028 = 16215

[745 176 128 828 537]

   blend_time blend_code   blend_desc  sequence  quantity   silo   line  \
0           3      B0002  Black Tea 2         1      1000  siloE  line6   

  sku_code  quantity_line  
0    F0002          18300  

         blend_code blend_description  quantity
location                                       
siloA         B0004       Loose Tea 1      5000
siloB         B0004       Loose Tea 1      4000
siloC         B0001   

# Iteration Process for Silo A, B

## Step 1

In [18]:
connection_matrix = pd.read_excel('connection_matrix4.xlsx')
connection_matrix = connection_matrix.set_index('sku_code')
connection_matrix

,siloA,siloB
sku_code,,
F0007,1,0
F0014,0,1


In [19]:
Iter0 = 0
Iter = solve_connection_matrix2(connection_matrix, Iter0)

iter = 1

Stock needed for F0007 = 5574

Stock needed for F0014 = 10258

[229 160]



iter = 2

Stock needed for F0007 = 4639

Stock needed for F0014 = 9188

[186 117]



iter = 3

Stock needed for F0007 = 3704

Stock needed for F0014 = 8118

[143  74]

    blend_time blend_code   blend_desc  sequence  quantity   silo   line  \
66          92      B0001  Black Tea 1         1      1000  siloC  line1   
67          95      B0001  Black Tea 1         1      1000  siloC  line1   
68          96      B0001  Black Tea 1         1      1000  siloD  line7   
69          99      B0001  Black Tea 1         1      1000  siloC  line1   
70           3      B0004  Loose Tea 1         1      1000  siloB  line4   

   sku_code  quantity_line  
66    F0001           2690  
67    F0001           1760  
68    F0006            695  
69    F0001            520  
70    F0014           8118  

         blend_code blend_description  quantity
location                                       
siloA         B000

## Step 2

In [20]:
for i in range(2, 6):
    print()
    print('==================================================================================================================================================================')
    print()
    
    if connection_matrix.iloc[0]['siloA'] == 0:
        sku_code_old = connection_matrix.index[0]
        sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
        silo = 'siloA'
        connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
        connection_matrix.at[sku_code_new, silo] = 1
        print(connection_matrix)
        print()
        print('=================================================================================')
        print()
        
        # do they use different blend_code ? If yes, empty the silo
        if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
            blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
            blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']
            
            # Send to warehouse
            wastage = wastage + blend_stock1.loc[silo]['quantity']
            blend_stock1.at[silo, 'quantity'] = 0
            
            print(blend_stock1)
            print()
            print('=================================================================================')
            print()
            
    elif connection_matrix.iloc[1]['siloB'] == 0:
        sku_code_old = connection_matrix.index[1]
        sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
        silo = 'siloB'
        connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
        connection_matrix.at[sku_code_new, silo] = 1
        print(connection_matrix)
        print()
        print('=================================================================================')
        print()
        
        # do they use different blend_code ? If yes, empty the silo
        if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
            blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
            blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']
            
            # Send to warehouse
            wastage = wastage + blend_stock1.loc[silo]['quantity']
            blend_stock1.at[silo, 'quantity'] = 0
            
            print(blend_stock1)
            print()
            print('=================================================================================')
            print()
        
    Iter = solve_connection_matrix2(connection_matrix, Iter)



          siloA  siloB
sku_code              
F0008         1      0
F0014         0      1


iter = 9

Stock needed for F0008 = 24469

Stock needed for F0014 = 3328

[22  0]

    blend_time blend_code   blend_desc  sequence  quantity   silo   line  \
72           5      B0004  Loose Tea 1         1      1000  siloA  line5   
73           6      B0004  Loose Tea 1         1      1000  siloB  line4   
74           7      B0004  Loose Tea 1         1      1000  siloA  line5   
75           8      B0004  Loose Tea 1         1      1000  siloB  line4   
76           9      B0004  Loose Tea 1         1      1000  siloB  line4   

   sku_code  quantity_line  
72    F0007           1834  
73    F0014           5328  
74    F0007            509  
75    F0014           4328  
76    F0014           3328  

         blend_code blend_description  quantity
location                                       
siloA         B0004       Loose Tea 1         0
siloB         B0004       Loose Tea 1      100

## Step 3

In [21]:
connection_matrix

,siloA,siloB
sku_code,,
F0009,1,0
F0016,0,1


In [22]:
Iter = solve_connection_matrix1(connection_matrix, Iter)

Iter = 82

Stock needed for F0009 = 10337

Stock needed for F0016 = 10800

[45  0]




Iter = 83

Stock needed for F0009 = 10272

Stock needed for F0016 = 10800

[2 0]




Iter = 84

Stock needed for F0009 = 10272

Stock needed for F0016 = 10800



## Step 4

In [23]:
connection_matrix.at['F0009', 'siloA'] = 0
connection_matrix.at['F0016', 'siloB'] = 0
connection_matrix

,siloA,siloB
sku_code,,
F0009,0,0
F0016,0,0


In [24]:
if connection_matrix.iloc[0]['siloA'] == 0:
    sku_code_old = connection_matrix.index[0]
    sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
    silo = 'siloA'
    connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
    connection_matrix.at[sku_code_new, silo] = 1
    print(connection_matrix)
    print()
    print('=================================================================================')
    print()

    # do they use different blend_code ? If yes, empty the silo
    if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
        blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
        blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']

        # Send to warehouse
        wastage = wastage + blend_stock1.loc[silo]['quantity']
        blend_stock1.at[silo, 'quantity'] = 0

        print(blend_stock1)
        print()
        print('=================================================================================')
        print()

if connection_matrix.iloc[1]['siloB'] == 0:
    sku_code_old = connection_matrix.index[1]
    sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
    silo = 'siloB'
    connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
    connection_matrix.at[sku_code_new, silo] = 1
    print(connection_matrix)
    print()
    print('=================================================================================')
    print()

    # do they use different blend_code ? If yes, empty the silo
    if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
        blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
        blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']

        # Send to warehouse
        wastage = wastage + blend_stock1.loc[silo]['quantity']
        blend_stock1.at[silo, 'quantity'] = 0

        print(blend_stock1)
        print()
        print('=================================================================================')
        print()

          siloA  siloB
sku_code              
F0010         1      0
F0016         0      0


         blend_code blend_description  quantity
location                                       
siloA         B0006       Loose Tea 3         0
siloB         B0005       Loose Tea 2         0
siloC         B0001       Black Tea 1      1220
siloD         B0001       Black Tea 1       960
siloE         B0002       Black Tea 2      1113
siloF         B0003       Black Tea 3       355


          siloA  siloB
sku_code              
F0010         1      0
F0017         0      1


         blend_code blend_description  quantity
location                                       
siloA         B0006       Loose Tea 3         0
siloB         B0006       Loose Tea 3         0
siloC         B0001       Black Tea 1      1220
siloD         B0001       Black Tea 1       960
siloE         B0002       Black Tea 2      1113
siloF         B0003       Black Tea 3       355




In [25]:
for i in range(7, 11):
    print()
    print('==================================================================================================================================================================')
    print('i = ' + str(i))
    print()
    
    if connection_matrix.iloc[0]['siloA'] == 0:
        sku_code_old = connection_matrix.index[0]
        sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
        silo = 'siloA'
        connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
        connection_matrix.at[sku_code_new, silo] = 1
        print(connection_matrix)
        print()
        print('=================================================================================')
        print()
        
        # do they use different blend_code ? If yes, empty the silo
        if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
            blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
            blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']
            
            # Send to warehouse
            wastage = wastage + blend_stock1.loc[silo]['quantity']
            blend_stock1.at[silo, 'quantity'] = 0
            
            print(blend_stock1)
            print()
            print('=================================================================================')
            print()
            
    elif connection_matrix.iloc[1]['siloB'] == 0:
        sku_code_old = connection_matrix.index[1]
        sku_code_new = next_sku_code.loc[sku_code_old]['sku_code_next']
        silo = 'siloB'
        connection_matrix = connection_matrix.rename(index={sku_code_old:sku_code_new})
        connection_matrix.at[sku_code_new, silo] = 1
        print(connection_matrix)
        print()
        print('=================================================================================')
        print()
        
        # do they use different blend_code ? If yes, empty the silo
        if next_sku_code.loc[sku_code_old]['blend'] != next_sku_code.loc[sku_code_old]['blend_next']:
            blend_stock1.at[silo, 'blend_code'] = production_schedule.loc[sku_code_new]['blend_code']
            blend_stock1.at[silo, 'blend_description'] = production_schedule.loc[sku_code_new]['blend_description']
            
            # Send to warehouse
            wastage = wastage + blend_stock1.loc[silo]['quantity']
            blend_stock1.at[silo, 'quantity'] = 0
            
            print(blend_stock1)
            print()
            print('=================================================================================')
            print()
        
    Iter = solve_connection_matrix2(connection_matrix, Iter)


i = 7

iter = 84

Stock needed for F0010 = 8370

Stock needed for F0017 = 17820

[0 0]

     blend_time blend_code   blend_desc  sequence  quantity   silo   line  \
145          78      B0005  Loose Tea 2         3      1000  siloB  line4   
146          79      B0005  Loose Tea 2         3      1000  siloA  line5   
147          80      B0005  Loose Tea 2         3      1000  siloB  line4   
148          81      B0005  Loose Tea 2         3      1000  siloA  line5   
149          84      B0006  Loose Tea 3         4      1000  siloA  line5   

    sku_code  quantity_line  
145    F0016          12800  
146    F0009          12272  
147    F0016          11800  
148    F0009          11272  
149    F0010           8370  

         blend_code blend_description  quantity
location                                       
siloA         B0006       Loose Tea 3      1000
siloB         B0006       Loose Tea 3         0
siloC         B0001       Black Tea 1      1220
siloD         B0001       B

# Show Result and Data

In [26]:
blend_schedule = blend_schedule.sort_values(by=['blend_time'])
blend_schedule.index = range(blend_schedule.shape[0])
blend_schedule.to_excel('blend_schedule_result.xlsx', index=False)
blend_schedule

,blend_time,blend_code,blend_desc,sequence,quantity,silo,line,sku_code,quantity_line
0,3,B0002,Black Tea 2,1,1000,siloE,line6,F0002,18300
1,3,B0004,Loose Tea 1,1,1000,siloB,line4,F0014,8118
2,4,B0004,Loose Tea 1,1,1000,siloB,line4,F0014,7048
3,5,B0002,Black Tea 2,1,1000,siloE,line6,F0002,17540
4,5,B0004,Loose Tea 1,1,1000,siloA,line5,F0007,1834
...,...,...,...,...,...,...,...,...,...
189,124,B0007,Loose Tea 4,6,1000,siloA,line5,F0012,11500
190,125,B0006,Loose Tea 3,5,1000,siloB,line4,F0018,1060
191,126,B0007,Loose Tea 4,6,1000,siloA,line5,F0012,10500
192,127,B0006,Loose Tea 3,5,1000,siloB,line4,F0018,60


In [27]:
production_schedule

,line,description,blend_code,blend_description,quantity,sequence,time_expected,time_last
sku_code,,,,,,,,
F0001,line1,LYL 1,B0001,Black Tea 1,0,1,4267,4343
F0027,line2,ENV 3,B0012,Black Tea 7,1302,1,654,0
F0023,line3,ENV 1,B0009,Black Tea 4,233,1,117,0
F0024,line3,ENV 2,B0010,Black Tea 5,3726,2,1874,0
F0014,line4,LYL Loose 3,B0004,Loose Tea 1,0,1,454,0
F0015,line4,LYL Loose 4,B0004,Loose Tea 1,0,2,549,0
F0016,line4,BB Loose 6,B0005,Loose Tea 2,10800,3,1156,0
F0017,line4,BB Loose 7,B0006,Loose Tea 3,0,4,715,0
F0018,line4,BB Loose 8,B0006,Loose Tea 3,0,5,130,0


In [28]:
blend_stock1

,blend_code,blend_description,quantity
location,,,
siloA,B0007,Loose Tea 4,65
siloB,B0006,Loose Tea 3,940
siloC,B0001,Black Tea 1,1220
siloD,B0001,Black Tea 1,960
siloE,B0002,Black Tea 2,1113
siloF,B0003,Black Tea 3,355


In [29]:
rm_stock

,blend_code,blend_description,rm_description,quantity,location
rm_code,,,,,
R0001,B0001,Black Tea 1,Raw: Black Tea 1,22000,Warehouse
R0002,B0002,Black Tea 2,Raw: Black Tea 2,1000,Warehouse
R0003,B0003,Black Tea 3,Raw: Black Tea 3,37000,Warehouse
R0004,B0004,Loose Tea 1,Raw: Loose Tea 1,1000,Warehouse
R0005,B0005,Loose Tea 2,Raw: Loose Tea 2,0,Warehouse
R0006,B0006,Loose Tea 3,Raw: Loose Tea 3,5000,Warehouse
R0007,B0007,Loose Tea 4,Raw: Loose Tea 4,35000,Warehouse
R0008,B0008,Loose Tea 5,Raw: Loose Tea 5,15000,Warehouse
R0009,B0009,Black Tea 4,Raw: Black Tea 4,0,Warehouse


# Objectives

## Wastage

In [30]:
wastage = wastage + sum(blend_stock1['quantity'])
production_schedule_temp = production_schedule.loc[production_schedule['quantity'] == 0]
print('Wastage for ' + str(production_schedule_temp.shape[0]) + ' jobs / sku codes = ' + str(wastage) + ' kg.')

Wastage for 13 jobs / sku codes = 7166 kg.


## Idle Time

In [31]:
production_schedule_temp = production_schedule.loc[production_schedule['time_last'] > 0]
idle_time_average = np.mean(production_schedule_temp['time_last'] - production_schedule_temp['time_expected'])
print('Average idle time = ' + str(idle_time_average) + ' minutes.')

Average idle time = 103.2 minutes.


1. Muhammad Luthfi Shahab
2. luthfi.shahab@gmail.com
3. 100060503@ku.ac.ae